In [112]:
#import dependencies
import pandas as pd
import numpy as np
import os
import gmaps
import json
import requests
from pprint import pprint
import us

#import API key
from config import gkey, weather_api_key
gmaps.configure(api_key=gkey)

#output file path for data resource files and figures
outputFilePath = os.path.join("Output-Data")

In [91]:
#read the sunshine csv file into a dataframe
sunshine_df = pd.read_csv(os.path.join("..","potential_data_sets","NOAA_Sunshine_AveragePercentPossible.csv"))
sunshine_df

,CITY,STATE,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA
0,YUMA,AZ,90%,42
1,REDDING,CA,88%,10
2,LAS VEGAS,NV,85%,47
3,PHOENIX,AZ,85%,101
4,TUCSON,AZ,85%,53
...,...,...,...,...
169,"POHNPEI, MICRONESIA",PC,41%,44
170,ELKINS,WV,40%,11
171,MT. WASHINGTON,NH,33%,64
172,QUILLAYUTE,WA,33%,30


In [92]:
#check the datatypes
sunshine_df.dtypes

CITY                              object
STATE                             object
ANNUAL % AVG POSSIBLE SUNSHINE    object
NO. OF YEARS OF DATA               int64
dtype: object

In [93]:
#strip the % column of the % symbol and convert to float
sunshine_df["ANNUAL % AVG POSSIBLE SUNSHINE"] = sunshine_df["ANNUAL % AVG POSSIBLE SUNSHINE"].map(lambda x: x.rstrip('%'))
sunshine_df["ANNUAL % AVG POSSIBLE SUNSHINE"] = sunshine_df["ANNUAL % AVG POSSIBLE SUNSHINE"].astype(float)
sunshine_df

,CITY,STATE,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA
0,YUMA,AZ,90.0,42
1,REDDING,CA,88.0,10
2,LAS VEGAS,NV,85.0,47
3,PHOENIX,AZ,85.0,101
4,TUCSON,AZ,85.0,53
...,...,...,...,...
169,"POHNPEI, MICRONESIA",PC,41.0,44
170,ELKINS,WV,40.0,11
171,MT. WASHINGTON,NH,33.0,64
172,QUILLAYUTE,WA,33.0,30


In [94]:
#credit to: https://code.activestate.com/recipes/577305-python-dictionary-of-us-states-and-territories/
#dictionary of state abbreviations:
states_dict = {
        'AK': 'Alaska',
        'AL': 'Alabama',
        'AR': 'Arkansas',
        'AS': 'American Samoa',
        'AZ': 'Arizona',
        'CA': 'California',
        'CO': 'Colorado',
        'CT': 'Connecticut',
        'DC': 'District of Columbia',
        'DE': 'Delaware',
        'FL': 'Florida',
        'GA': 'Georgia',
        'GU': 'Guam',
        'HI': 'Hawaii',
        'IA': 'Iowa',
        'ID': 'Idaho',
        'IL': 'Illinois',
        'IN': 'Indiana',
        'KS': 'Kansas',
        'KY': 'Kentucky',
        'LA': 'Louisiana',
        'MA': 'Massachusetts',
        'MD': 'Maryland',
        'ME': 'Maine',
        'MI': 'Michigan',
        'MN': 'Minnesota',
        'MO': 'Missouri',
        'MP': 'Northern Mariana Islands',
        'MS': 'Mississippi',
        'MT': 'Montana',
        'NA': 'National',
        'NC': 'North Carolina',
        'ND': 'North Dakota',
        'NE': 'Nebraska',
        'NH': 'New Hampshire',
        'NJ': 'New Jersey',
        'NM': 'New Mexico',
        'NV': 'Nevada',
        'NY': 'New York',
        'OH': 'Ohio',
        'OK': 'Oklahoma',
        'OR': 'Oregon',
        'PA': 'Pennsylvania',
        'PR': 'Puerto Rico',
        'RI': 'Rhode Island',
        'SC': 'South Carolina',
        'SD': 'South Dakota',
        'TN': 'Tennessee',
        'TX': 'Texas',
        'UT': 'Utah',
        'VA': 'Virginia',
        'VI': 'Virgin Islands',
        'VT': 'Vermont',
        'WA': 'Washington',
        'WI': 'Wisconsin',
        'WV': 'West Virginia',
        'WY': 'Wyoming'
}

In [95]:
# #convert state to full state name

#displays 'AZ'
# sunshine_df['STATE'][0]

#errors out saying: module 'us.states' has no attribute 'sunshine_df'
# us.states.sunshine_df['STATE'][0].name

#displays 'Arizona'
# us.states.AZ.name

In [96]:
#JB's help/idea for a solution

#still didn't work, returning "module 'us.states' has no attribute 'short'" error
# shorts = sunshine_df['STATE']
# full_names = []
# for short in shorts:
#     full_name = us.states.short.name
#     full_names.append(full_name)

# full_names

In [97]:
#mannually clean the data set - we don't know how to web scrape yet
stateNames_df = pd.read_csv(os.path.join("..","potential_data_sets","50_us_states_all_data.csv"))
stateNames_df.head()

,CAPS STATE NAME,TITLE STATE NAME,2 LETTER ABBREVIATION,ABBREV.
0,ALABAMA,Alabama,AL,Ala.
1,ALASKA,Alaska,AK,Alaska
2,ARIZONA,Arizona,AZ,Ariz.
3,ARKANSAS,Arkansas,AR,Ark.
4,CALIFORNIA,California,CA,Calif.


In [138]:
#join the two dataframes
cleanSunshine_df = pd.merge(sunshine_df,stateNames_df,how='left',left_on="STATE",right_on="2 LETTER ABBREVIATION")
#clean up formatting
del cleanSunshine_df['STATE']
del cleanSunshine_df['CAPS STATE NAME']
del cleanSunshine_df['2 LETTER ABBREVIATION']
del cleanSunshine_df['ABBREV.']
cleanSunshine_df = cleanSunshine_df.rename(columns={"TITLE STATE NAME":"STATE"})
# cleanSunshine_df = cleanSunshine_df.dropna().reset_index(drop=True) #filtering these out later - google geocoding can find most of the territories still
cleanSunshine_df

,CITY,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA,STATE
0,YUMA,90.0,42,Arizona
1,REDDING,88.0,10,California
2,LAS VEGAS,85.0,47,Nevada
3,PHOENIX,85.0,101,Arizona
4,TUCSON,85.0,53,Arizona
...,...,...,...,...
169,"POHNPEI, MICRONESIA",41.0,44,NaN
170,ELKINS,40.0,11,West Virginia
171,MT. WASHINGTON,33.0,64,New Hampshire
172,QUILLAYUTE,33.0,30,Washington


In [139]:
#google maps geocoding url
url_geocoding = "https://maps.googleapis.com/maps/api/geocode/json?"

#find the latitude and longitude of the cities
lat = []
lng = []

#define countrycode
currentCountry = "US"

#loop through the cities_df to retrieve the weather information
for index, row in cleanSunshine_df.iterrows():
    currentCity = row["CITY"]
    currentState = row["STATE"]
    target_url = f"{url_geocoding}address={currentCity},{currentState}&region={currentCountry}&key={gkey}"
    response = requests.get(target_url).json()
    
    try:
        lat.append(response['results'][0]['geometry']['location']['lat'])
        lng.append(response['results'][0]['geometry']['location']['lng'])
        print(f"Processeing Record | {currentCity}\n{query_url}")
    except:
        lat.append(np.nan)
        lng.append(np.nan)
        print(f"City not found. Skipping {currentCity}...\n{query_url}")

#update latitude and longitude to actually match the city
cleanSunshine_df["Latitude"] = lat
cleanSunshine_df["Longitude"] = lng

Processeing Record | YUMA
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | REDDING
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | LAS VEGAS
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | PHOENIX
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | TUCSON
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | EL PASO
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | FRESNO
http://api.openweathermap.org/data/2.5/weather?q=SAN ANT

Processeing Record | HURON
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | JACKSONVILLE
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | RAPID CITY
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | RICHMOND
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | SHERIDAN
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | WILMINGTON
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | CHARLOTTE
http://api.openweathermap.org/data/2.5/w

http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | PROVIDENCE
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | RALEIGH
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | SPRINGFIELD
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | BALTIMORE
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | BIRMINGHAM
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | CHATTANOOGA
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas

Processeing Record | SAULT STE. MARIE
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | SEATTLE SEA-TAC AP
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | GRAND RAPIDS
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | GUAM
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | SYRACUSE
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | PITTSBURGH
http://api.openweathermap.org/data/2.5/weather?q=SAN ANTONIO,Texas,US&units=imperial&appid=ede721544ba8261f052c5133e7a7ee14
Processeing Record | SEATTLE
http://api.openweathermap.o

In [140]:
#export results to csv file
cleanSunshine_df.to_csv(os.path.join(outputFilePath,"cleanSunshine.csv"),index=False)

cleanSunshine_df

,CITY,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA,STATE,Latitude,Longitude
0,YUMA,90.0,42,Arizona,32.692651,-114.627692
1,REDDING,88.0,10,California,40.586540,-122.391675
2,LAS VEGAS,85.0,47,Nevada,36.169941,-115.139830
3,PHOENIX,85.0,101,Arizona,33.448377,-112.074037
4,TUCSON,85.0,53,Arizona,32.222607,-110.974711
...,...,...,...,...,...,...
169,"POHNPEI, MICRONESIA",41.0,44,NaN,6.844570,158.335640
170,ELKINS,40.0,11,West Virginia,38.925940,-79.846735
171,MT. WASHINGTON,33.0,64,New Hampshire,44.270585,-71.303272
172,QUILLAYUTE,33.0,30,Washington,47.943130,-124.542435


In [135]:
#clean the data set of null values
sunshineLocations_df = cleanSunshine_df.dropna().reset_index(drop=True)
sunshineLocations_df

,CITY,ANNUAL % AVG POSSIBLE SUNSHINE,NO. OF YEARS OF DATA,STATE,Latitude,Longitude
0,YUMA,90.0,42,Arizona,32.692651,-114.627692
1,REDDING,88.0,10,California,40.586540,-122.391675
2,LAS VEGAS,85.0,47,Nevada,36.169941,-115.139830
3,PHOENIX,85.0,101,Arizona,33.448377,-112.074037
4,TUCSON,85.0,53,Arizona,32.222607,-110.974711
...,...,...,...,...,...,...
156,HILO,41.0,52,Hawaii,19.724111,-155.086819
157,ELKINS,40.0,11,West Virginia,38.925940,-79.846735
158,MT. WASHINGTON,33.0,64,New Hampshire,44.270585,-71.303272
159,QUILLAYUTE,33.0,30,Washington,47.943130,-124.542435


In [136]:
#collect the weightsof %
sunshine = sunshineLocations_df['ANNUAL % AVG POSSIBLE SUNSHINE'].astype(float)

#max of % sunshine
maxSun = sunshineLocations_df['ANNUAL % AVG POSSIBLE SUNSHINE'].max()

#locations of sun for heat
locations = sunshineLocations_df[["Latitude","Longitude"]].astype(float)

In [137]:
#create the sunshine map
fig = gmaps.figure()

#create the heat layer
heatLayer_sunshine = gmaps.heatmap_layer(locations, weights=sunshine, max_intensity=maxSun, dissipating=True)

#add the heat layer
fig.add_layer(heatLayer_sunshine)

fig

Figure(layout=FigureLayout(height='420px'))